<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/Tokyo_Marathon_Bib_Mosaic_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
bghai19_tokiomarathon2020_path = kagglehub.dataset_download('bghai19/tokiomarathon2020')

print('Data source import complete.')


# Tokyo Marathon Bib Mosaic YOLOv8
https://docs.ultralytics.com/tasks/pose/
https://www.kaggle.com/code/stpeteishii/tokyo-marathon-pose-yolov8

In [ ]:
!rm -rf images
!rm -rf runs
!rm -rf yolov8_pose
!rm *

In [ ]:
!ls

In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from IPython.display import Video
from ultralytics import YOLO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
!mkdir images

In [ ]:
paths0=[]
for dirname, _, filenames in os.walk('/kaggle/input/tokiomarathon2020'):
    for filename in filenames:
        paths0+=[(os.path.join(dirname, filename))]

dest='./images'
for i,path in enumerate(paths0[3:8]):
    shutil.copy(path,os.path.join(dest,f'{i:02d}.png'))

!ls images

In [ ]:
paths=[]
for dirname, _, filenames in os.walk('./images'):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]

paths.sort()
print(paths)

    Argument	Type	Default	Description
    
    show	bool	False	If True, displays the annotated images or videos in a window. Useful for immediate visual feedback during development or testing.
    
    save	bool	False or True	Enables saving of the annotated images or videos to file. Useful for documentation, further analysis, or sharing results. Defaults to True when using CLI & False when used in Python.
    
    save_frames	bool	False	When processing videos, saves individual frames as images. Useful for extracting specific frames or for detailed frame-by-frame analysis.
    
    save_txt	bool	False	Saves detection results in a text file, following the format [class] [x_center] [y_center] [width] [height] [confidence]. Useful for integration with other analysis tools.
    
    save_conf	bool	False	Includes confidence scores in the saved text files. Enhances the detail available for post-processing and analysis.
    
    save_crop	bool	False	Saves cropped images of detections. Useful for dataset augmentation, analysis, or creating focused datasets for specific objects.
    
    show_labels	bool	True	Displays labels for each detection in the visual output. Provides immediate understanding of detected objects.
    
    show_conf	bool	True	Displays the confidence score for each detection alongside the label. Gives insight into the model's certainty for each detection.
    
    show_boxes	bool	True	Draws bounding boxes around detected objects. Essential for visual identification and location of objects in images or video frames.
    
    line_width	None or int	None	Specifies the line width of bounding boxes. If None, the line width is automatically adjusted based on the image size. Provides visual customization for clarity.

In [ ]:
!yolo pose predict model=yolov8x-pose.pt source=images show_boxes=False save_txt=True conf=0.8

In [ ]:
output_path = '/kaggle/working/runs/pose/predict'

opaths=[]
tpaths=[]
for dirname, _, filenames in os.walk(output_path):
    for filename in filenames:
        path=(os.path.join(dirname, filename))
        if filename.endswith('.jpg'):
            opaths+=[path]
        else:
            tpaths+=[path]

opaths.sort()
tpaths.sort()
print(opaths)
print(tpaths)

In [ ]:
# [class_id] [x_center] [y_center] [width] [height] [keypoint_1_x] [keypoint_1_y] [keypoint_1_conf] ... [keypoint_n_x] [keypoint_n_y] [keypoint_n_conf]

for path in tpaths:
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)

In [ ]:
def parse_pose_data(lines):
    """
    Function to extract position information of each body part from multiple lines of YOLOv8 Pose output.
    Args:
        lines (list of str): A list of output lines from YOLOv8.
    Returns:
        list of dict: A list of dictionaries, each containing body part positions for a line of output.
                      If a body part is not detected, its value will be None.
    """
    # COCO body part names (17-point format)
    keypoint_names = [
        "nose", "left_eye", "right_eye", "left_ear", "right_ear",
        "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
        "left_wrist", "right_wrist", "left_hip", "right_hip",
        "left_knee", "right_knee", "left_ankle", "right_ankle"
    ]

    # Result list to store the parsed data for each line
    parsed_results = []

    for line in lines.splitlines():

        # Parse the data from each line
        data = list(map(float, line.strip().split()))

        # Class ID (the first value)
        class_id = int(data[0])

        # Bounding box (next 4 values)
        bbox = {
            "x_center": data[1],
            "y_center": data[2],
            "width": data[3],
            "height": data[4]
        }

        # Body part information (from the 5th value onward)
        keypoints = data[5:]

        # Convert keypoint information to a dictionary
        result = {name: None for name in keypoint_names}
        keypoint_index = 0

        for name in keypoint_names:
            if keypoint_index < len(keypoints):
                # If not detected, the data will be "0"
                if keypoints[keypoint_index] == 0:
                    result[name] = None
                    keypoint_index += 3  # Skip one set (for undetected keypoints)
                else:
                    # If detected, store the (x, y, confidence)
                    x = keypoints[keypoint_index]
                    y = keypoints[keypoint_index + 1]
                    conf = keypoints[keypoint_index + 2]
                    result[name] = (x, y, conf)
                    keypoint_index += 3  # Move ahead by three values (x, y, confidence)

        # Append the result for this line
        parsed_results.append({"class_id": class_id, "bbox": bbox, "keypoints": result})

    return parsed_results

In [ ]:
def calculate_bounding_box(keypoints, part1, part2):
    """
    Function to calculate a bounding box based on two body parts.
    Args:
        keypoints (dict): A dictionary with body part names as keys and (x, y, confidence) as values.
        part1 (str): The name of the first body part to be used in the bounding box.
        part2 (str): The name of the second body part to be used in the bounding box.
    Returns:
        dict: A dictionary representing the bounding box (x_min, y_min, x_max, y_max) or None if either part is not detected.
    """
    # Retrieve the coordinates for part1 and part2
    part1_data = keypoints.get(part1)
    part2_data = keypoints.get(part2)

    # If either part is not detected (None), return None
    if part1_data is None:
        print(f"{part1} is not detected.")
        return None
    if part2_data is None:
        print(f"{part2} is not detected.")
        return None

    # Extract coordinates (x, y)
    x1, y1, _ = part1_data
    x2, y2, _ = part2_data

    # Calculate the bounding box by finding the min/max x and y values
    x_min = min(x1, x2)
    y_min = min(y1, y2)
    x_max = max(x1, x2)
    y_max = max(y1, y2)

    return {"x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}

In [ ]:
RESULTS=[]
for path in tpaths:
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
        results = parse_pose_data(content)
        RESULTS+=[results]
        print(len(results))

In [ ]:
BOX=[]
for results in RESULTS:
    boxes=[]
    for result in results:
        # Retrieve Keypoints
        keypoints = result["keypoints"]

        # Bounding box for left shoulder and right hip
        box_left_shoulder_right_hip = calculate_bounding_box(keypoints, "left_shoulder", "right_hip")
        if box_left_shoulder_right_hip is not None:
            print("Bounding box for left shoulder and right hip:", box_left_shoulder_right_hip)
            boxes+=[box_left_shoulder_right_hip]
        else:
            print("Bounding box for left shoulder and right hip not detected.")

        # Bounding box for right shoulder and left hip
        box_right_shoulder_left_hip = calculate_bounding_box(keypoints, "right_shoulder", "left_hip")
        if box_right_shoulder_left_hip is not None:
            print("Bounding box for right shoulder and left hip:", box_right_shoulder_left_hip)
            boxes+=[box_right_shoulder_left_hip]
        else:
            print("Bounding box for right shoulder and left hip not detected.")
    BOX+=[boxes]

In [ ]:
def get_middle_bbox(x0, y0, x1, y1):
    """
    Given a bounding box (x0, y0, x1, y1), this function divides it into three equal parts along the y-axis
    and returns the coordinates of the middle part of the bounding box.

    Args:
        x0 (float): The x-coordinate of the top-left corner of the bounding box.
        y0 (float): The y-coordinate of the top-left corner of the bounding box.
        x1 (float): The x-coordinate of the bottom-right corner of the bounding box.
        y1 (float): The y-coordinate of the bottom-right corner of the bounding box.

    Returns:
        tuple: The coordinates of the middle bounding box (x0_new, y0_new, x1_new, y1_new).
    """
    # Height of the bounding box
    height = max(y0,y1) - min(y0,y1)

    # Calculate the middle position by dividing the height into three equal parts
    middle_start = int(y0 + height / 3)
    middle_end = int(y0 + 2 * (height / 3))

    # Return the coordinates of the new bounding box
    return (x0 - 5, middle_start, x1 + 5, middle_end)

In [ ]:
def mosaic(src, ratio=0.1):
    small = cv2.resize(src, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    return cv2.resize(small, src.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

In [ ]:
for i,boxes in enumerate(BOX):
    path=paths[i]
    image2=plt.imread(path)
    H,W=image2.shape[0:2]
    print(image2.flatten().max())

    for box in boxes:
        try:
            x0, y0, x1, y1 = int(box['x_min']*W),int(box['y_min']*H),int(box['x_max']*W),int(box['y_max']*H)
            x2, y2, x3, y3 = get_middle_bbox(x0, y0, x1, y1)
            #print(x2,y2,x3,y3)
            #cv2.rectangle(image2,(x2,y2),(x3,y3),(255,0,0),3)
            mosaic_image=mosaic(image2[y2:y3,x2:x3])
            image2[y2:y3,x2:x3]=mosaic_image
        except:
            pass

    output_path = f"./mz{path.split('/')[-1]}"
    cv2.imwrite(output_path, image2*255)

    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(image2)
    plt.show()

In [ ]:
from PIL import Image
frames = [Image.open(path) for path in opaths]
frames[0].save(
    "animation.gif",
    save_all=True,
    append_images=frames[1:],
    duration=1000,
    loop=0
)
output_path="animation.gif"

In [ ]:
from IPython.display import Image
Image(open(output_path, 'rb').read())